In [9]:
import spotipy
import pandas as pd
import numpy as np
from spotipy.oauth2 import SpotifyOAuth
import time

In [26]:
client_id='c4ff820f994745288df451ea7d6940d5'
client_secret='be2297719e5e4fb698f112c4374ee3b1'
redirect_uri='https://open.spotify.com/'
token_url = "https://accounts.spotify.com/api/token"

## User Auth
- access spotify api via OAuth workflow

In [15]:
scope = 'playlist-read-private'

sp_oauth = SpotifyOAuth(client_id=client_id,client_secret=client_secret,redirect_uri=redirect_uri,scope=scope)
token_info = sp_oauth.get_cached_token() 
if not token_info:
    auth_url = sp_oauth.get_authorize_url()
    print(auth_url)
    response = input('Paste the above link into your browser, then paste the redirect url here: ')

    code = sp_oauth.parse_response_code(response)
    token_info = sp_oauth.get_access_token(code)

    token = token_info['access_token']

sp = spotipy.Spotify(auth=token, retries=10, requests_timeout=10)

https://accounts.spotify.com/authorize?client_id=c4ff820f994745288df451ea7d6940d5&response_type=code&redirect_uri=https%3A%2F%2Fopen.spotify.com%2F&scope=playlist-read-private


/var/folders/dy/m7tb4mx52sv0lxfvtf95srbc0000gn/T/ipykernel_78378/4118829141.py:11: DeprecationWarning: You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.
  token_info = sp_oauth.get_access_token(code)


In [16]:
# scope = 'playlist-read-private'

# oauth = spotipy.util.prompt_for_user_token(client_id=client_id, client_secret=client_secret, redirect_uri=redirect_uri, scope=scope)
# sp = spotipy.Spotify(auth=oauth)

In [17]:
if sp:
    print(sp.me())

{'display_name': 'amaya lim', 'external_urls': {'spotify': 'https://open.spotify.com/user/nightrainlily'}, 'href': 'https://api.spotify.com/v1/users/nightrainlily', 'id': 'nightrainlily', 'images': [{'url': 'https://i.scdn.co/image/ab67757000003b82180bd1d0c567a1e26710aeb8', 'height': 64, 'width': 64}, {'url': 'https://i.scdn.co/image/ab6775700000ee85180bd1d0c567a1e26710aeb8', 'height': 300, 'width': 300}], 'type': 'user', 'uri': 'spotify:user:nightrainlily', 'followers': {'href': None, 'total': 364}}


## Getting Playlists
- get all playlists via current_user_playlist method. use offsets to avoid rate limiting
- filter playlists using regex for only project playlists (numbered)
- create pandas df for analysis

In [31]:
from time import sleep
from tqdm import tqdm, trange

In [46]:
def get_all_playlists():
    playlist_count = 1000
    results = []
    offset = 0
    pbar = tqdm(total=1000)
    while offset < playlist_count:
        results.append(sp.current_user_playlists(limit=50, offset=offset))
        time.sleep(10)
        pbar.update(50)
        offset += 50
    pbar.close()
    return results
results = get_all_playlists()











































100%|██████████| 1000/1000 [03:30<00:00,  4.74it/s]


In [47]:
def parse(playlist_json, target):
    lst = []
    for i in np.arange(len(playlist_json['items'])):
        target_value = playlist_json['items'][i]['track'][target]
        lst.append(target_value)
    return lst 

In [48]:
import re

playlist_data = []

for result in results:
    # pbar = tqdm(total=1000)
    for playlist in result['items']:
        reg = "^([0-9])+\s([a-z]+(\s?)([a-z]?))"
        # pbar.update(1)
        if re.search(reg, playlist['name']) != None:
            track_info = sp.playlist_tracks(playlist['id'], fields='items(track(id, name))')
            playlist_data.append({
                'name' : playlist['name'],
                'track_ids' : parse(track_info, 'id'),
                'track_names' : parse(track_info, 'name'),
                'id' : playlist['id'],
                'image' : playlist['images'][0]['url']
            })
    # pbar.close()

playlists = pd.DataFrame(playlist_data)
# playlists.head().style.format({'image': lambda x : f'<img src="{x}" width="100px"/>'})
# playlists.head()

In [49]:
def pretty_tracks(col, name):
        all_l = []
        for i in np.arange(len(playlists[col])):
                row = playlists[col].values[i]
                l = []
                for j in row:
                        for k in np.arange(len(row)):
                                track = item[k]['track']
                                id = track[name]
                                l.append(id)
                all_l.append(l)
        return all_l

# playlists['track_ids'] = pretty_tracks('track_ids', 'id')
# playlists['track_names'] = pretty_tracks('track_names', 'name')

# playlists.head().style.format({'image': lambda x : f'<img src="{x}" width="100px"/>'})
playlists.head()


,name,track_ids,track_names,id,image
0,00 another dream,"[0uQo0HbOwXdPRGlNyFqGKi, 3UY4s7F4Ard3T6vm7xcJx...","[You Never Knew, Haircut, Touching Yourself, S...",538ZY5ILCTT6PxqytGgkYH,https://image-cdn-ak.spotifycdn.com/image/ab67...
1,842 what did you expect,"[0rxQTgJzd0HCUk4QUxSkjK, 1EvFahLuLZ0vSVZo8YR3G...","[Even If It Hurts (feat. Blood Orange), Due We...",0KgwFInw8xWYmtDQWayf2p,https://mosaic.scdn.co/640/ab67616d00001e02145...
2,841 promise i'm better,"[12UQIJePnGeLmpVReYpG2w, 6ELP6nqqRDftftpjgioiW...","[Sunny day, Getting Older, Mutations, Le Templ...",6OrcGRAMqQ3A6o2zCmqTj5,https://mosaic.scdn.co/640/ab67616d00001e02384...
3,840 it falls apart,"[10cGyorRXoOc9RhVhY9Sru, 2TPy9mk2KVIwSXrqwgKk2...","[She's Leaving You, Warmest Part of the Winter...",2rARwXlp3rvRwTDnT6KXXu,https://mosaic.scdn.co/640/ab67616d00001e02012...
4,840 pretty woman,"[35tWhD29yvWwB0IDRr6zsL, 5sMEEjviCkH6Rp5X2ZvII...","[Yeah Right, Everything is romantic, Frank's T...",6T6cMAfk5OTqGcs3L1gUUU,https://image-cdn-ak.spotifycdn.com/image/ab67...


In [25]:
def get_url(p_name):
    info = sp.playlist(playlists[playlists['name'] == p_name]['id'].item())
    url = info['external_urls']['spotify']
    return url

'https://open.spotify.com/playlist/538ZY5ILCTT6PxqytGgkYH'

## Getting Song Data
- manipulate playlist df to get songs
- drop na values (locally stored files)

In [57]:
songs = playlists
songs['zipp'] = list(zip(songs['track_ids'], songs['track_names']))

In [58]:
def merge(z):
    return list(tuple(zip(z[0], z[1])))

songs['zipp'] = songs['zipp'].apply(merge)
songs.head()

,name,track_ids,track_names,id,image,zipp
0,00 another dream,"[0uQo0HbOwXdPRGlNyFqGKi, 3UY4s7F4Ard3T6vm7xcJx...","[You Never Knew, Haircut, Touching Yourself, S...",538ZY5ILCTT6PxqytGgkYH,https://image-cdn-ak.spotifycdn.com/image/ab67...,"[(0uQo0HbOwXdPRGlNyFqGKi, You Never Knew), (3U..."
1,842 what did you expect,"[0rxQTgJzd0HCUk4QUxSkjK, 1EvFahLuLZ0vSVZo8YR3G...","[Even If It Hurts (feat. Blood Orange), Due We...",0KgwFInw8xWYmtDQWayf2p,https://mosaic.scdn.co/640/ab67616d00001e02145...,"[(0rxQTgJzd0HCUk4QUxSkjK, Even If It Hurts (fe..."
2,841 promise i'm better,"[12UQIJePnGeLmpVReYpG2w, 6ELP6nqqRDftftpjgioiW...","[Sunny day, Getting Older, Mutations, Le Templ...",6OrcGRAMqQ3A6o2zCmqTj5,https://mosaic.scdn.co/640/ab67616d00001e02384...,"[(12UQIJePnGeLmpVReYpG2w, Sunny day), (6ELP6nq..."
3,840 it falls apart,"[10cGyorRXoOc9RhVhY9Sru, 2TPy9mk2KVIwSXrqwgKk2...","[She's Leaving You, Warmest Part of the Winter...",2rARwXlp3rvRwTDnT6KXXu,https://mosaic.scdn.co/640/ab67616d00001e02012...,"[(10cGyorRXoOc9RhVhY9Sru, She's Leaving You), ..."
4,840 pretty woman,"[35tWhD29yvWwB0IDRr6zsL, 5sMEEjviCkH6Rp5X2ZvII...","[Yeah Right, Everything is romantic, Frank's T...",6T6cMAfk5OTqGcs3L1gUUU,https://image-cdn-ak.spotifycdn.com/image/ab67...,"[(35tWhD29yvWwB0IDRr6zsL, Yeah Right), (5sMEEj..."


In [59]:
songs = songs.explode('zipp')
songs.head()

,name,track_ids,track_names,id,image,zipp
0,00 another dream,"[0uQo0HbOwXdPRGlNyFqGKi, 3UY4s7F4Ard3T6vm7xcJx...","[You Never Knew, Haircut, Touching Yourself, S...",538ZY5ILCTT6PxqytGgkYH,https://image-cdn-ak.spotifycdn.com/image/ab67...,"(0uQo0HbOwXdPRGlNyFqGKi, You Never Knew)"
0,00 another dream,"[0uQo0HbOwXdPRGlNyFqGKi, 3UY4s7F4Ard3T6vm7xcJx...","[You Never Knew, Haircut, Touching Yourself, S...",538ZY5ILCTT6PxqytGgkYH,https://image-cdn-ak.spotifycdn.com/image/ab67...,"(3UY4s7F4Ard3T6vm7xcJxQ, Haircut)"
0,00 another dream,"[0uQo0HbOwXdPRGlNyFqGKi, 3UY4s7F4Ard3T6vm7xcJx...","[You Never Knew, Haircut, Touching Yourself, S...",538ZY5ILCTT6PxqytGgkYH,https://image-cdn-ak.spotifycdn.com/image/ab67...,"(1HfsmNlg8xxhOr9N2i4Q0n, Touching Yourself)"
0,00 another dream,"[0uQo0HbOwXdPRGlNyFqGKi, 3UY4s7F4Ard3T6vm7xcJx...","[You Never Knew, Haircut, Touching Yourself, S...",538ZY5ILCTT6PxqytGgkYH,https://image-cdn-ak.spotifycdn.com/image/ab67...,"(5ksM8r8u4AjMwvtqUif9W1, Slide Tackle)"
0,00 another dream,"[0uQo0HbOwXdPRGlNyFqGKi, 3UY4s7F4Ard3T6vm7xcJx...","[You Never Knew, Haircut, Touching Yourself, S...",538ZY5ILCTT6PxqytGgkYH,https://image-cdn-ak.spotifycdn.com/image/ab67...,"(2IM2fs9rVuDbqtuXpycvam, rangerover - Bonus Tr..."


In [60]:
songs['track_name'] = songs['zipp'].map(lambda x: x[1])
songs['track_id'] = songs['zipp'].map(lambda x: x[0])
songs.head()

,name,track_ids,track_names,id,image,zipp,track_name,track_id
0,00 another dream,"[0uQo0HbOwXdPRGlNyFqGKi, 3UY4s7F4Ard3T6vm7xcJx...","[You Never Knew, Haircut, Touching Yourself, S...",538ZY5ILCTT6PxqytGgkYH,https://image-cdn-ak.spotifycdn.com/image/ab67...,"(0uQo0HbOwXdPRGlNyFqGKi, You Never Knew)",You Never Knew,0uQo0HbOwXdPRGlNyFqGKi
0,00 another dream,"[0uQo0HbOwXdPRGlNyFqGKi, 3UY4s7F4Ard3T6vm7xcJx...","[You Never Knew, Haircut, Touching Yourself, S...",538ZY5ILCTT6PxqytGgkYH,https://image-cdn-ak.spotifycdn.com/image/ab67...,"(3UY4s7F4Ard3T6vm7xcJxQ, Haircut)",Haircut,3UY4s7F4Ard3T6vm7xcJxQ
0,00 another dream,"[0uQo0HbOwXdPRGlNyFqGKi, 3UY4s7F4Ard3T6vm7xcJx...","[You Never Knew, Haircut, Touching Yourself, S...",538ZY5ILCTT6PxqytGgkYH,https://image-cdn-ak.spotifycdn.com/image/ab67...,"(1HfsmNlg8xxhOr9N2i4Q0n, Touching Yourself)",Touching Yourself,1HfsmNlg8xxhOr9N2i4Q0n
0,00 another dream,"[0uQo0HbOwXdPRGlNyFqGKi, 3UY4s7F4Ard3T6vm7xcJx...","[You Never Knew, Haircut, Touching Yourself, S...",538ZY5ILCTT6PxqytGgkYH,https://image-cdn-ak.spotifycdn.com/image/ab67...,"(5ksM8r8u4AjMwvtqUif9W1, Slide Tackle)",Slide Tackle,5ksM8r8u4AjMwvtqUif9W1
0,00 another dream,"[0uQo0HbOwXdPRGlNyFqGKi, 3UY4s7F4Ard3T6vm7xcJx...","[You Never Knew, Haircut, Touching Yourself, S...",538ZY5ILCTT6PxqytGgkYH,https://image-cdn-ak.spotifycdn.com/image/ab67...,"(2IM2fs9rVuDbqtuXpycvam, rangerover - Bonus Tr...",rangerover - Bonus Track,2IM2fs9rVuDbqtuXpycvam


In [61]:
songs.reset_index()

,index,name,track_ids,track_names,id,image,zipp,track_name,track_id
0,0,00 another dream,"[0uQo0HbOwXdPRGlNyFqGKi, 3UY4s7F4Ard3T6vm7xcJx...","[You Never Knew, Haircut, Touching Yourself, S...",538ZY5ILCTT6PxqytGgkYH,https://image-cdn-ak.spotifycdn.com/image/ab67...,"(0uQo0HbOwXdPRGlNyFqGKi, You Never Knew)",You Never Knew,0uQo0HbOwXdPRGlNyFqGKi
1,0,00 another dream,"[0uQo0HbOwXdPRGlNyFqGKi, 3UY4s7F4Ard3T6vm7xcJx...","[You Never Knew, Haircut, Touching Yourself, S...",538ZY5ILCTT6PxqytGgkYH,https://image-cdn-ak.spotifycdn.com/image/ab67...,"(3UY4s7F4Ard3T6vm7xcJxQ, Haircut)",Haircut,3UY4s7F4Ard3T6vm7xcJxQ
2,0,00 another dream,"[0uQo0HbOwXdPRGlNyFqGKi, 3UY4s7F4Ard3T6vm7xcJx...","[You Never Knew, Haircut, Touching Yourself, S...",538ZY5ILCTT6PxqytGgkYH,https://image-cdn-ak.spotifycdn.com/image/ab67...,"(1HfsmNlg8xxhOr9N2i4Q0n, Touching Yourself)",Touching Yourself,1HfsmNlg8xxhOr9N2i4Q0n
3,0,00 another dream,"[0uQo0HbOwXdPRGlNyFqGKi, 3UY4s7F4Ard3T6vm7xcJx...","[You Never Knew, Haircut, Touching Yourself, S...",538ZY5ILCTT6PxqytGgkYH,https://image-cdn-ak.spotifycdn.com/image/ab67...,"(5ksM8r8u4AjMwvtqUif9W1, Slide Tackle)",Slide Tackle,5ksM8r8u4AjMwvtqUif9W1
4,0,00 another dream,"[0uQo0HbOwXdPRGlNyFqGKi, 3UY4s7F4Ard3T6vm7xcJx...","[You Never Knew, Haircut, Touching Yourself, S...",538ZY5ILCTT6PxqytGgkYH,https://image-cdn-ak.spotifycdn.com/image/ab67...,"(2IM2fs9rVuDbqtuXpycvam, rangerover - Bonus Tr...",rangerover - Bonus Track,2IM2fs9rVuDbqtuXpycvam
...,...,...,...,...,...,...,...,...,...
9692,926,007 all this time // taste for the relevant,"[1DrlLvlYd1FIjNavRm6NdX, 15jNKKCmLjyGQnKzphBiO...","[Back Pocket, Front Tooth, Does It Feel Real, ...",2Gg4bWhMij2PXtI6AGvPPS,https://mosaic.scdn.co/640/ab67616d00001e0216a...,"(4YZbVct8l9MnAVIROnLQdx, Forrest Gump)",Forrest Gump,4YZbVct8l9MnAVIROnLQdx
9693,926,007 all this time // taste for the relevant,"[1DrlLvlYd1FIjNavRm6NdX, 15jNKKCmLjyGQnKzphBiO...","[Back Pocket, Front Tooth, Does It Feel Real, ...",2Gg4bWhMij2PXtI6AGvPPS,https://mosaic.scdn.co/640/ab67616d00001e0216a...,"(0DXlHWi8pqB5R2gfzlZx2i, No Better)",No Better,0DXlHWi8pqB5R2gfzlZx2i
9694,926,007 all this time // taste for the relevant,"[1DrlLvlYd1FIjNavRm6NdX, 15jNKKCmLjyGQnKzphBiO...","[Back Pocket, Front Tooth, Does It Feel Real, ...",2Gg4bWhMij2PXtI6AGvPPS,https://mosaic.scdn.co/640/ab67616d00001e0216a...,"(4M2t7bP4Mq87mGMn0PObUX, untitled 06 | 06.30.2...",untitled 06 | 06.30.2014.,4M2t7bP4Mq87mGMn0PObUX
9695,926,007 all this time // taste for the relevant,"[1DrlLvlYd1FIjNavRm6NdX, 15jNKKCmLjyGQnKzphBiO...","[Back Pocket, Front Tooth, Does It Feel Real, ...",2Gg4bWhMij2PXtI6AGvPPS,https://mosaic.scdn.co/640/ab67616d00001e0216a...,"(0ekkYaWO7OR9RBWtOcGPKM, Herringbone)",Herringbone,0ekkYaWO7OR9RBWtOcGPKM


In [71]:
songs = songs.dropna()

### Audio Features
- manipulate song df to get audio features from spotify api
- merge with song df for searchability
- take mean audio features as score for playlist overall
- merge with playlist df
- export dfs as csvs to avoid repeatedly calling api for analysis

In [80]:
def batch(lst, n):
    for i in range(0, len(lst), n):
        if len(lst) >= (i+n):
            yield lst[i:i + n]
        else:
            yield lst[i:]

def get_audio_features(uris):
    results = []
    pbar = tqdm(total=1000)
    batched = batch(uris, 100)
    for each in batched:
        results.append(sp.audio_features(each))
        pbar.update(50)
    pbar.close()
    return results

In [81]:
audio_features_results = get_audio_features(songs['track_id'])







































































































































































































































































































4850it [00:23, 203.02it/s]


In [309]:
unpacked = [song for hundred in audio_features_results for song in hundred]

In [310]:
features = pd.DataFrame(unpacked)
features.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.738,0.707,7,-5.966,1,0.0265,0.1430,0.000010,0.1050,0.808,117.999,audio_features,0uQo0HbOwXdPRGlNyFqGKi,spotify:track:0uQo0HbOwXdPRGlNyFqGKi,https://api.spotify.com/v1/tracks/0uQo0HbOwXdP...,https://api.spotify.com/v1/audio-analysis/0uQo...,269227,4
1,0.779,0.495,8,-7.621,1,0.0381,0.7310,0.000332,0.1320,0.546,128.992,audio_features,3UY4s7F4Ard3T6vm7xcJxQ,spotify:track:3UY4s7F4Ard3T6vm7xcJxQ,https://api.spotify.com/v1/tracks/3UY4s7F4Ard3...,https://api.spotify.com/v1/audio-analysis/3UY4...,216003,4
2,0.714,0.680,3,-5.989,1,0.0260,0.1350,0.000002,0.2290,0.934,108.001,audio_features,1HfsmNlg8xxhOr9N2i4Q0n,spotify:track:1HfsmNlg8xxhOr9N2i4Q0n,https://api.spotify.com/v1/tracks/1HfsmNlg8xxh...,https://api.spotify.com/v1/audio-analysis/1Hfs...,181587,4
3,0.746,0.603,5,-6.516,1,0.0297,0.0178,0.013900,0.0573,0.874,127.989,audio_features,5ksM8r8u4AjMwvtqUif9W1,spotify:track:5ksM8r8u4AjMwvtqUif9W1,https://api.spotify.com/v1/tracks/5ksM8r8u4AjM...,https://api.spotify.com/v1/audio-analysis/5ksM...,219783,4
4,0.634,0.654,4,-5.351,0,0.0547,0.0584,0.000000,0.1990,0.782,149.782,audio_features,2IM2fs9rVuDbqtuXpycvam,spotify:track:2IM2fs9rVuDbqtuXpycvam,https://api.spotify.com/v1/tracks/2IM2fs9rVuDb...,https://api.spotify.com/v1/audio-analysis/2IM2...,135703,4


In [311]:
songs_features = songs.merge(features,  left_on='track_id', right_on='id')
songs_features.head()

,name,track_ids,track_names,id_x,image,zipp,track_name,track_id,danceability,energy,...,liveness,valence,tempo,type,id_y,uri,track_href,analysis_url,duration_ms,time_signature
0,00 another dream,"[0uQo0HbOwXdPRGlNyFqGKi, 3UY4s7F4Ard3T6vm7xcJx...","[You Never Knew, Haircut, Touching Yourself, S...",538ZY5ILCTT6PxqytGgkYH,https://image-cdn-ak.spotifycdn.com/image/ab67...,"(0uQo0HbOwXdPRGlNyFqGKi, You Never Knew)",You Never Knew,0uQo0HbOwXdPRGlNyFqGKi,0.738,0.707,...,0.105,0.808,117.999,audio_features,0uQo0HbOwXdPRGlNyFqGKi,spotify:track:0uQo0HbOwXdPRGlNyFqGKi,https://api.spotify.com/v1/tracks/0uQo0HbOwXdP...,https://api.spotify.com/v1/audio-analysis/0uQo...,269227,4
1,00 another dream,"[0uQo0HbOwXdPRGlNyFqGKi, 3UY4s7F4Ard3T6vm7xcJx...","[You Never Knew, Haircut, Touching Yourself, S...",538ZY5ILCTT6PxqytGgkYH,https://image-cdn-ak.spotifycdn.com/image/ab67...,"(3UY4s7F4Ard3T6vm7xcJxQ, Haircut)",Haircut,3UY4s7F4Ard3T6vm7xcJxQ,0.779,0.495,...,0.132,0.546,128.992,audio_features,3UY4s7F4Ard3T6vm7xcJxQ,spotify:track:3UY4s7F4Ard3T6vm7xcJxQ,https://api.spotify.com/v1/tracks/3UY4s7F4Ard3...,https://api.spotify.com/v1/audio-analysis/3UY4...,216003,4
2,00 another dream,"[0uQo0HbOwXdPRGlNyFqGKi, 3UY4s7F4Ard3T6vm7xcJx...","[You Never Knew, Haircut, Touching Yourself, S...",538ZY5ILCTT6PxqytGgkYH,https://image-cdn-ak.spotifycdn.com/image/ab67...,"(3UY4s7F4Ard3T6vm7xcJxQ, Haircut)",Haircut,3UY4s7F4Ard3T6vm7xcJxQ,0.779,0.495,...,0.132,0.546,128.992,audio_features,3UY4s7F4Ard3T6vm7xcJxQ,spotify:track:3UY4s7F4Ard3T6vm7xcJxQ,https://api.spotify.com/v1/tracks/3UY4s7F4Ard3...,https://api.spotify.com/v1/audio-analysis/3UY4...,216003,4
3,00 another dream,"[0uQo0HbOwXdPRGlNyFqGKi, 3UY4s7F4Ard3T6vm7xcJx...","[You Never Knew, Haircut, Touching Yourself, S...",538ZY5ILCTT6PxqytGgkYH,https://image-cdn-ak.spotifycdn.com/image/ab67...,"(1HfsmNlg8xxhOr9N2i4Q0n, Touching Yourself)",Touching Yourself,1HfsmNlg8xxhOr9N2i4Q0n,0.714,0.680,...,0.229,0.934,108.001,audio_features,1HfsmNlg8xxhOr9N2i4Q0n,spotify:track:1HfsmNlg8xxhOr9N2i4Q0n,https://api.spotify.com/v1/tracks/1HfsmNlg8xxh...,https://api.spotify.com/v1/audio-analysis/1Hfs...,181587,4
4,00 another dream,"[0uQo0HbOwXdPRGlNyFqGKi, 3UY4s7F4Ard3T6vm7xcJx...","[You Never Knew, Haircut, Touching Yourself, S...",538ZY5ILCTT6PxqytGgkYH,https://image-cdn-ak.spotifycdn.com/image/ab67...,"(1HfsmNlg8xxhOr9N2i4Q0n, Touching Yourself)",Touching Yourself,1HfsmNlg8xxhOr9N2i4Q0n,0.714,0.680,...,0.229,0.934,108.001,audio_features,1HfsmNlg8xxhOr9N2i4Q0n,spotify:track:1HfsmNlg8xxhOr9N2i4Q0n,https://api.spotify.com/v1/tracks/1HfsmNlg8xxh...,https://api.spotify.com/v1/audio-analysis/1Hfs...,181587,4


In [312]:
playlist_features = songs_features[['name', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']].groupby('name').median()
playlist_features.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
name,,,,,,,,,,,
00 all yours,0.583,0.569,7.0,-7.9775,1.0,0.03600,0.3700,0.000694,0.1110,0.5520,114.7475
00 alright,0.612,0.702,2.0,-5.7070,1.0,0.05280,0.0908,0.000022,0.0806,0.5020,122.8040
00 another dream,0.714,0.630,5.0,-6.3450,1.0,0.03630,0.1350,0.000007,0.1320,0.7775,115.9975
00 before you go,0.518,0.341,5.0,-9.0890,1.0,0.04115,0.6270,0.000000,0.1105,0.2330,124.1130
00 call it love,0.551,0.758,2.0,-7.3850,1.0,0.03520,0.0878,0.000099,0.0960,0.5470,118.0200


In [156]:
playlists.to_csv('csv_files/playlists.csv', index=False)
songs.to_csv('csv_files/songs.csv', index=False)
features.to_csv('csv_files/features.csv', index=False)

In [523]:
playlist_features.to_csv('csv_files/playlist_features.csv', index=True)